# *This file allows to generate all plots necessary for figures*

# **Useful packages and functions**

In [12]:
using DifferentialEquations, Plots, Polynomials, LaTeXStrings, ColorSchemes, DelimitedFiles, DataFrames
using Statistics, StatsPlots, Random, ProgressMeter, Printf, LinearAlgebra, Plots.PlotMeasures
include("DA_kinetics.jl") # Loading of DA kinetics of gating variables
include("DA_models.jl") # Loading of DA model
include("DA_utils.jl"); # Loading of some utils functions

# **Global variables**

In [13]:
# Definition of simulation time (in ms)
const Tfinal = 20000
const tspan  = (0.0, Tfinal)
tt = 0. : 0.01 : Tfinal
tt_rand = 0. : 1 : Tfinal

# Definition of reversal potential values (in mV), [Mg] and membrane capacitance
const VNa     = 60. # Sodium reversal potential
const VK      = -90. # Potassium reversal potential
const VCa     = 50. # Calcium reversal potential
const VH      = -29. # H reversal potential
const VLNS    = -65. # Leak reversal potential
const EPacemaker = 4.2732015978991615 # Reversal potential of pacemaking channels

const C       = 1. # Membrane capacitance
const fCa     = 0.018 # Fraction of unbuffered free calcium
const ICapmax = 11 # Maximum calcium pump current
const F       = 96520 # Faraday constant in ms*µA/mmol (and taking cm³=mL)
const d       = 15 # Soma diameter in cm
const L       = 25 # Soma length

# Definition of voltage range for the DICs
const Vmin = -100 
const Vmax = 50
const Vrange = range(Vmin, stop=Vmax, step=0.0154640);

In [14]:
# Modifying backend GR attributes
gr(guidefontsize=25, tickfontsize=15, legendfontsize=12, margin=5Plots.mm, grid=false)
myApple = RGBA(187/255, 206/255, 131/255, 1)
mySalmon = RGBA(243/255, 124/255, 130/255)
myYellow = RGBA(228/255, 205/255, 121/255, 1)
myBlue = RGBA(131/255, 174/255, 218/255, 1)
myDarkBlue = RGBA(114/255, 119/255, 217/255, 1)
myOrange = RGBA(241/255, 175/255, 113/255, 1)
myPink = RGBA(243/255, 124/255, 130/255, 1)
myPurple = RGBA(169/255, 90/255, 179/255, 1)
myGreen = RGBA(132/255, 195/255, 168/255, 1)
myRed = RGBA(158/255, 3/255, 8/255, 1)
myGray = RGBA(150/255, 150/255, 150/255, 1)
myLightBlue = RGBA(127/255, 154/255, 209/255, 1);
default(fmt = :png);

In [15]:
# Define a struct (optional, but useful if you need parameters)
struct NoisyFunction
    amplitude::Float64  # amplitude of the noise
end

# Overload the () operator to make the struct callable
function (nf::NoisyFunction)(x::Float64)
    noise = nf.amplitude * randn()  # Generate Gaussian noise (mean 0, std 1)
    return noise  # Example function with noise
end

function condition(u,t,integrator) # Event when event_f(u,t) == 0
  (u[1]- (-20.))
end

function affect!(integrator)
end

cb = ContinuousCallback(condition, affect!, nothing, save_positions = (true, false));

# Simu original model

In [10]:
# i = 0
# # Initializing some variables
# N = 200

# # Input current definition
# Iapp(t) = 0 # pA

# # Initial conditions
# V0 = -50.
# Ca0 = 1e-4
# x0 = [V0, m_inf(V0), h_inf(V0), hs_inf(V0), l_inf(V0), n_inf(V0), p_inf(V0), q1_inf(V0), q2_inf(V0), 
#     0., 0., mH_inf(V0), Ca0]

# n_neurons = 0
# while n_neurons < N
#     display("Trial n")
#     i = i + 1
#     display(i)
#     gNa        = 100*rand()[1] # Sodium current maximal conductance
#     gCaL       = 5*rand()[1] # L-type calcium current maximal conductance
#     gKd        = 50*rand()[1] # Delayed-rectifier potassium current maximal conductance
#     gKA        = 5*rand()[1] # A-type potassium current maximal conductance
#     gKERG      = 1*rand()[1] # ERG current maximal conductance
#     gKSK       = 2*rand()[1] # SK current maximal conductance
#     gH         = 0.5*rand()[1] # H current maximal conductance
#     gLNS       = 0.1*rand()[1] # Leak non specific current maximal conductance
#     gLCa       = 0.1*rand()[1] # Leak calcium current maximal conductance
#     gPacemaker = 75*rand()[1] # Pacemaker current maximal conductance
#     p = (Iapp, gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa, gPacemaker)

#     # Simulation
#     prob = ODEProblem(DA_ODE_true_instant, x0, tspan, p) # Describing the problem
#     sol = solve(prob, Rodas5(), maxtime=30.0; maxiters=1e10); # Solving the problem
    
#     if sol.t[end] > Tfinal - 10
#         x        = sol(tt)
#         V_sim    = x[1, :][1000000:end]
        
#         ISIs_i = extract_ISI(V_sim, tt[1000000:end])
#     else
#         continue
#     end

#     if length(ISIs_i) < 20
#         continue
#     else
#         if maximum(ISIs_i[10:end]) - minimum(ISIs_i[10:end]) > 50
#             continue
#         end

#         f = 1000/mean(ISIs_i[10:end])

#         if f < 5 && f > 1 && maximum(V_sim) < 30 && maximum(V_sim) > 0 && minimum(V_sim) < -60 && minimum(V_sim) > -90
#             n_neurons = n_neurons + 1
#             display(n_neurons)
#             display(f)
#             display([gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa, gPacemaker])
            
#             filename = "./data/g_all_pacemaking$(n_neurons).dat"
#             writedlm(filename, [gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa, gPacemaker])
#         end
#     end
# end

"Trial n"

1

"Trial n"

2

"Trial n"

...

"Trial n"

4299

200

2.1796425023022477

10-element Vector{Float64}:
 40.096587178207685
  4.468888667749582
 38.1962704712815
  1.1076353481917462
  0.3287382714255066
  1.573012443415309
  0.25512585091816636
  0.048967039408082225
  0.0017406291207923141
  8.311746599383591

In [13]:
# N = 200
# g_all_pacemaking = zeros(N, 10)
# for i = 1 : N
#     filename = "./data/g_all_pacemaking$(i).dat"
#     g_all_pacemaking[i, :] = vec(readdlm(filename))
# end

# writedlm("./data/g_all_pacemaker.dat", g_all_pacemaking);

In [11]:
i = 0
# Initializing some variables
N = 94

# Input current definition
Iapp(t) = 0 # pA

# Initial conditions
V0 = -50.
Ca0 = 1e-4
x0 = [V0, m_inf(V0), h_inf(V0), hs_inf(V0), l_inf(V0), n_inf(V0), p_inf(V0), q1_inf(V0), q2_inf(V0), 
    0., 0., mH_inf(V0), Ca0]

n_neurons = 92
while n_neurons < N
    display("Trial n")
    i = i + 1
    display(i)
    gNa        = 15*rand()[1] # Sodium current maximal conductance
    gCaL       = 1.5*rand()[1] # L-type calcium current maximal conductance
    gKd        = 50*rand()[1] # Delayed-rectifier potassium current maximal conductance
    gKA        = 5*rand()[1] # A-type potassium current maximal conductance
    gKERG      = 1*rand()[1] # ERG current maximal conductance
    gKSK       = 2*rand()[1] # SK current maximal conductance
    gH         = 0.5*rand()[1] # H current maximal conductance
    gLNS       = 0.1*rand()[1] # Leak non specific current maximal conductance
    gLCa       = 0.1*rand()[1] # Leak calcium current maximal conductance
    p = (Iapp, gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa)

    # Simulation
    prob = ODEProblem(DA_ODE, x0, tspan, p) # Describing the problem
    sol = solve(prob, Rodas5(), maxtime=30.0; maxiters=1e10); # Solving the problem
    
    if sol.t[end] > Tfinal - 10
        x        = sol(tt)
        V_sim    = x[1, :][1000000:end]
        
        ISIs_i = extract_ISI(V_sim, tt[1000000:end])
    else
        continue
    end

    if length(ISIs_i) < 20
        continue
    else
        if maximum(ISIs_i[10:end]) - minimum(ISIs_i[10:end]) > 50
            continue
        end

        f = 1000/mean(ISIs_i[10:end])

        if f < 5 && f > 1 && maximum(V_sim) < 30 && maximum(V_sim) > 0 && minimum(V_sim) < -60 && minimum(V_sim) > -90
            n_neurons = n_neurons + 1
            display(n_neurons)
            display(f)
            display([gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa])
            
            filename = "./data/g_all$(n_neurons).dat"
            writedlm(filename, [gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa])
        end
    end
end

"Trial n"

1

"Trial n"

2

"Trial n"

...

"Trial n"

80

94

4.730653226497285

9-element Vector{Float64}:
  4.478827463893475
  0.1135330133628073
 22.537650918923514
  1.016274702542805
  0.829165734402055
  0.0589419924417085
  0.04024626185842495
  0.06222476251357613
  0.00045256119205667745

In [17]:
N = 200
g_all = zeros(N, 9)
for i = 1 : N
    filename = "./data/g_all$(i).dat"
    g_all[i, :] = vec(readdlm(filename))
end

writedlm("./data/g_all.dat", g_all);

In [39]:
# dt = 0.0005
# t_Euler = 0 : dt : Tfinal

# # Input current definition
# Iapp(t) = 0 # pA

# # Initial conditions
# V0 = -50.
# Ca0 = 1e-4
# x0 = [V0, m_inf(V0), h_inf(V0), hs_inf(V0), l_inf(V0), n_inf(V0), p_inf(V0), q1_inf(V0), q2_inf(V0), 
#     0., 0., mH_inf(V0), Ca0]

In [40]:
# plot(t_Euler, V_Euler, xlims=(15000, 20000))

In [41]:
# gNa   = 4.627934541430234 # Sodium current maximal conductance
# gCaL  = 0.19089990490483966 # L-type calcium current maximal conductance
# gKd   = 18.173588625897935 # Delayed-rectifier potassium current maximal conductance
# gKA   = 2.6888854085463105 # A-type potassium current maximal conductance
# gKERG = 0.009572356228376053 # ERG current maximal conductance
# gKSK  = 0.1272853932290964 # SK current maximal conductance
# gH    = 0.11504286350280934 # H current maximal conductance
# gLNS  = 0.002954976929594939 # Leak non specific current maximal conductance
# gLCa  = 0.004627539588829168 # Leak calcium current maximal conductance

# # Input current definition
# Iapp(t) = 0 # pA
# p = (Iapp, gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa)

# V_Euler = simu_Euler(p, dt, x0)
    
# V_plot = V_Euler[10000000:end]

In [ ]:
# # Initializing some variables
# N = 48
# g_all = zeros(N, 9)

# dt = 0.0005
# t_Euler = 0 : dt : Tfinal

# # Input current definition
# Iapp(t) = 0 # pA

# # Initial conditions
# V0 = -50.
# Ca0 = 1e-4
# x0 = [V0, m_inf(V0), h_inf(V0), hs_inf(V0), l_inf(V0), n_inf(V0), p_inf(V0), q1_inf(V0), q2_inf(V0), 
#     0., 0., mH_inf(V0), Ca0]

# n_neurons = 47
# while n_neurons < N
#     gNa        = 25. *2*rand()[1] # Sodium current maximal conductance
#     gCaL       = 1. *2*rand()[1] # L-type calcium current maximal conductance
#     gKd        = 10. *2*rand()[1] # Delayed-rectifier potassium current maximal conductance
#     gKA        = 1.68 *2*rand()[1] # A-type potassium current maximal conductance
#     gKERG      = 0.13 *2*rand()[1] # ERG current maximal conductance
#     gKSK       = 0.3 *2*rand()[1] # SK current maximal conductance
#     gH         = 0.078 *2*rand()[1] # H current maximal conductance
#     gLNS       = 0.01 *2*rand()[1] # Leak non specific current maximal conductance
#     gLCa       = 0.00245 *2*rand()[1] # Leak calcium current maximal conductance
#     p = (Iapp, gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa)

#     V_Euler = simu_Euler(p, dt, x0)
    
#     V_plot = V_Euler[10000000:end]

#     ISIs_i = extract_ISI(V_Euler, t_Euler)

#     if length(ISIs_i) < 20
#         continue
#     else
        
#         if maximum(ISIs_i[10:end]) - minimum(ISIs_i[10:end]) > 50
#             continue
#         end

#         f = 1000/mean(ISIs_i[10:end])

#         if f < 5 && f > 1 && maximum(V_plot) < 30 && maximum(V_plot) > 0 && minimum(V_plot) < -60 && minimum(V_plot) > -90
#             n_neurons = n_neurons + 1
#             display(n_neurons)
#             display(f)
#             display([gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa])
            
#             filename = "./data/g_all$(n_neurons).dat"
#             writedlm(filename, [gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa])
            
#             g_all[n_neurons, :] = [gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa]
#         end
#     end
# end

In [ ]:
# # Initializing some variables
# N = 200
# g_all = zeros(N, 9)

# # Input current definition
# Iapp(t) = 0 # pA

# # Initial conditions
# V0 = -50.
# Ca0 = 1e-4
# x0 = [V0, m_inf(V0), h_inf(V0), hs_inf(V0), l_inf(V0), n_inf(V0), p_inf(V0), q1_inf(V0), q2_inf(V0), 
#     0., 0., mH_inf(V0), Ca0]

# n_neurons = 0
# while n_neurons < N
#     display("Still there")
#     gNa        = 25. *2*rand()[1] # Sodium current maximal conductance
#     gCaL       = 1. *2*rand()[1] # L-type calcium current maximal conductance
#     gKd        = 10. *2*rand()[1] # Delayed-rectifier potassium current maximal conductance
#     gKA        = 1.68 *2*rand()[1] # A-type potassium current maximal conductance
#     gKERG      = 0.13 *2*rand()[1] # ERG current maximal conductance
#     gKSK       = 0.3 *2*rand()[1] # SK current maximal conductance
#     gH         = 0.078 *2*rand()[1] # H current maximal conductance
#     gLNS       = 0.01 *2*rand()[1] # Leak non specific current maximal conductance
#     gLCa       = 0.00245 *2*rand()[1] # Leak calcium current maximal conductance
#     p = (Iapp, gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa)

#     # Simulation
#     prob = ODEProblem(DA_ODE, x0, tspan, p) # Describing the problem
#     sol = solve(prob; maxiters=1e6, verbose=false); # Solving the problem
#     sol_spike_times = solve(prob; callback=cb, save_everystep=false,save_start=false,save_end=false, verbose=false)

#     x         = sol(tt)
#     V_plot    = x[1, :]

#     if length(sol_spike_times.t) < 20
#         continue
#     else
#         # Extracting ISIs
#         spike_times = sol_spike_times.t
#         filter!(x -> x ≥ 5000, spike_times)

#         if length(spike_times) < 5
#             continue
#         end
        
#         # Extracting ISIs
#         ISIs = zeros(length(spike_times)-1)
#         for k = 1 : length(spike_times) - 1
#             ISIs[k] = spike_times[k+1] - spike_times[k]
#         end

#         f = 1000/mean(ISIs)

#         if f < 5 && f > 1 && maximum(V_plot) < 30 && maximum(V_plot) > 10 && minimum(V_plot) < -70 && minimum(V_plot) > -90
#             n_neurons = n_neurons + 1
#             display(n_neurons)
#             display(f)
#             display([gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa])
            
#             filename = "./data/g_all$(n_neurons).dat"
#             writedlm(filename, [gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa])
            
#             g_all[n_neurons, :] = [gNa, gCaL, gKd, gKA, gKERG, gKSK, gH, gLNS, gLCa]

#             voltage = plot(tt./1e3, V_plot, linewidth=2.5, color=myDarkBlue, xlims=(19., 20.),
#                 legend=false, ylims=(-100, 30), margins=20Plots.px, size=(1200, 800))
#             ylabel!("V (mV)")

#             display(voltage)
#         end
#     end
# end

In [ ]:
# writedlm(g_all, "./data/g_all.dat");

In [ ]:
# function simu_Euler(p, dt, x0)
#     t_Euler = 0 : dt : Tfinal
#     dx = zeros(size(x0))
#     V_Euler = zeros(length(t_Euler))
#     x = deepcopy(x0)
#     V_Euler[1] = x[1]
    
#     for i = 1 : length(t_Euler) - 1
#         DA_ODE(dx, x, p, t_Euler[i])
#         x .= x .+ dt * dx
#         V_Euler[i+1] = x[1]
#     end
    
#     return V_Euler
# end

In [ ]:
# function simu_Euler_pacemaking(p, dt, x0)
#     t_Euler = 0 : dt : Tfinal
#     dx = zeros(size(x0))
#     V_Euler = zeros(length(t_Euler))
#     x = deepcopy(x0)
#     V_Euler[1] = x[1]
    
#     for i = 1 : length(t_Euler) - 1
#         DA_ODE_true_instant(dx, x, p, t_Euler[i])
#         x .= x .+ dt * dx
#         V_Euler[i+1] = x[1]
#     end
    
#     return V_Euler
# end

In [ ]:
# dt = 0.0005
# t_Euler = 0 : dt : Tfinal;